In [1]:
import sciopy as scpy

Could not import module: serial


In [2]:
scpy.__all__

['print_syntax',
 'print_general_system_messages',
 'print_acknowledge_messages',
 'print_command_list',
 'available_serial_ports',
 'connect_COM_port']